In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import copy

from scripts.queso_model import model_data, model_vars, objective_func, balance, alloc_df

## Parámetros

In [3]:
# Escenario 1
folder = './data/test1/' # Carpeta con los datos
info_acopios = 'nuevo_info_acopios.xlsx' # Datos de centros de acopio 
costo_transporte = 'costoTransporteCAi.xlsx' # Matriz de costos de transporte entre CAs 
tiempo_transporte = 'tiempoTransporteCAi.xlsx' # Matriz de tiempos de transporte entre CAs
demanda = 60 # Demanda del cliente
ctiempo = 100 # Costo por unidad de tiempo

feromona_inicial = 1e-2
alpha = 1
beta = 1
rho = 0.999
Q = 1e3
tam_hormiguero = 30
max_iter = 300

In [5]:
archivos = {
    'info_acopios': info_acopios,
    'costo_transporte' : costo_transporte,
    'tiempo_transporte' : tiempo_transporte,
}

data = model_data(archivos, demanda, ctiempo, folder=folder)

demanda = data['demanda']
ctiempo = data['ctiempo']
# t_max = data['t_max']
params_df = data['params_df']

N, seed, capacidades = model_vars(params_df)

In [6]:
class CA:  # Centros de acopio
    def __init__(self, idCA, cant_stock, cant_pot):
        self.idCA = idCA
        self.en_stock = cant_stock  # True: disponible en stock, si no, es capacidad potencial
        self.potencial = cant_pot

Modelo de hormiga

$$
    p^{k}_{ij}(t) = 
        \frac{ [\tau_{ij}(t)]^\alpha \cdot [\eta_{ij}(t)]^\beta }{ \sum_{l \in J_i^k}[\tau_{ij}(t)]^\alpha \cdot [\eta_{il}]^\beta }
$$

In [ ]:
class Ant:
    def __init__(self, demanda, CAs_list, i, Q):
        self.ant_id = i
        self.sol_CAs = []
        self.cheese_cant = 0
        self.available_CAs = copy.deepcopy(CAs_list)
        self.demanda = demanda
        self.CAP = 0
        self.idx_CAP = 0
        self.eta_CAP = 0
        self.costo_total = 0
        self.pheromone_delta = np.zeros(len(self.available_CAs))
        self.Q = Q
        self.tao_ini = Q/Q
        
    
    def select_CAP(self, df_infoCA, costo_tiempo, alpha, beta):
        deltica = 1.e-5  # Para evitar división por 0
        tao = self.tao_ini  # Asumiendo feromona constante hasta el momento
        temp = np.zeros(len(self.available_CAs))
        denominador = 0
        i = 0
        
        for CA in self.available_CAs:
            if CA.en_Stock > 0:
                # Cantidad para asignar
                to_take = min(CA.en_Stock, self.demanda)
                eta = to_take * df_infoCA.loc[CA.idCA, 'Precio'] + df_infoCA.loc[CA.idCA, 'Ctransp'] + df_infoCA.loc[CA.idCA, 'TiempoTransp'] * costo_tiempo
                
                eta_beta = eta ** beta
                tao_alpha = tao ** alpha
                
                temp[i] = (eta_beta * tao_alpha)
                denominador += temp[i]
                
            i += 1
            
        probabilities = np.divide(temp, denominador + deltica)
        
        ids = 0
        rand = np.random.random()
        
        for i, prob in enumerate(probabilities):
            rand -= prob
            if rand <= 0:
                ids = i
                break
                
        self.CAP = self.available_CAs[ids].idCA
        self.cheese_cant = min(self.available_CAs[ids].en_Stock, self.demanda)
        self.available_CAs[ids].en_Stock = self.available_CAs[ids].en_Stock - self.cheese_cant
        self.idx_CAP = ids
        self.sol_CAs.append((self.CAP, self.cheese_cant, 'Stock'))
        self.eta_CAP = self.cheese_cant * df_infoCA.loc[self.CAP, 'Precio'] + df_infoCA.loc[self.CAP, 'Ctransp'] + df_infoCA.loc[self.CAP, 'TiempoTransp'] * costo_tiempo
        self.costo_total = self.eta_CAP
        
        
    def select_next_CA(self, df_infoCA, df_cTransp, df_tTransp, costo_tiempo, alpha, beta, pheromone):
        prob_temp = np.zeros(len(self.available_CAs))
        idx = 0
        eta = 0
        denominador = 0
        
        for CA in self.available_CAs:
            if CA.en_Stock + CA.potencial > 0:
                if CA.en_Stock > 0:
                    to_take = min(CA.en_Stock, self.demanda - self.cheese_cant)
                    eta = to_take * df_infoCA.loc[CA.idCA, 'Precio'] + df_cTransp.loc[self.CAP, CA.idCA] + df_tTransp.loc[self.CAP, CA.idCA] * costo_tiempo
                    
                elif CA.potencial > 0:
                    to_take = min(CA.potencial, self.demanda - self.cheese_cant)
                    eta = to_take * df_infoCA.loc[CA.idCA, 'Precio'] + df_infoCA.loc[CA.idCA, 'TiempoAlistam']
                        
                eta_beta = (eta + self.costo_total) ** beta
                tao_alpha = pheromone[self.idx_CAP][idx] ** alpha
                prob_temp[idx] = (eta_beta * tao_alpha)
                denominador += prob_temp[idx]
                
            idx += 1
        
        if denominador == 0: return -1
        probabilities = np.divide(prob_temp, denominador)
        
        ids = 0
        rand = np.random.random()
        for i, prob in enumerate(probabilities):
            rand -= prob
            if rand <= 0:
                ids = i
                break
                
        if self.available_CAs[ids].en_Stock + self.available_CAs[ids].potencial == 0:
            print('No hay queso disponible para asignar')
            return -1
        
        taken = 0
        if self.available_CAs[ids].en_Stock > 0:
            tipo = 'Stock'
            taken = min(self.available_CAs[ids].en_Stock, self.demanda - self.cheese_cant)
            self.cheese_cant += taken
            CA = self.available_CAs[ids].idCA
            self.available_CAs[ids].en_Stock = self.available_CAs[ids].en_Stock - taken
            costo = taken * df_infoCA.loc[CA, 'Precio'] + df_infoCA.loc[CA, 'TiempoAlistam'] * costo_tiempo
            
            if taken > 0:
                self.costo_total += costo
                self.sol_CAs.append((CA, taken, tipo))
                self.pheromone_delta[ids] += Q / costo
            return ids


    def complet_order(self, df_infoCA, df_cTransp, df_tTransp, costo_tiempo, beta, alpha, pheromone):
        while self.cheese_cant < self.demanda :
            self.select_next_CA(df_infoCA, df_cTransp, df_tTransp, costo_tiempo, alpha, beta, pheromone)


    def show_availableCAs(self):
        for CA in self.available_CAs:
            print(CA.idCA, CA.cantidad, CA.en_Stock)  